#### We will compare performance and accuracy aspects of "NaiveBayes" and "LogisticRegression"  algorithms within Titanic Dataset.


In [1]:
#load package
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
## Read in file
train_original = pd.read_csv('./data/train.csv')
test_original = pd.read_csv('./data/test.csv')
train_original.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_original.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


#### Preparing Data

In [4]:
#Creating seperate train sets for each ML Algorithm. 
df_logreg_train = train_original.copy()
df_logreg_test = test_original.copy()

#Dropping name, passengerId and ticket columns.
df_logreg_test.drop(["PassengerId", "Ticket","Name"], axis=1, inplace=True)
df_logreg_train.drop(["PassengerId", "Ticket","Name"], axis=1, inplace=True)
#Missing age fields are filled with mean
df_logreg_train["Age"].fillna(df_logreg_train.Age.median(), inplace=True)
df_logreg_test["Age"].fillna(df_logreg_test.Age.median(),inplace=True)
#Missing Fare and Cabin values are also filled
df_logreg_train["Fare"].fillna(df_logreg_train.Fare.median(), inplace=True)
df_logreg_train["Embarked"].fillna(df_logreg_train.Embarked.mode()[0], inplace=True)
df_logreg_test["Fare"].fillna(df_logreg_test.Fare.median(), inplace=True)
df_logreg_test["Embarked"].fillna(df_logreg_test.Embarked.mode()[0], inplace=True)
df_logreg_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,34.5,0,0,7.8292,NaN,Q
1,3,female,47.0,1,0,7.0000,NaN,S
2,2,male,62.0,0,0,9.6875,NaN,Q
3,3,male,27.0,0,0,8.6625,NaN,S
4,3,female,22.0,1,1,12.2875,NaN,S


Cabin has too many empty fields and it is not good idea to fill it with "median" modes. We will fill NaN fields with "Missing" instead, symbolized with M.

In [5]:
df_logreg_train["Cabin"].fillna('M', inplace=True)
df_logreg_test["Cabin"].fillna('M', inplace=True)
df_logreg_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,M,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,M,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,M,S


In [6]:
df_logreg_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,34.5,0,0,7.8292,M,Q
1,3,female,47.0,1,0,7.0000,M,S
2,2,male,62.0,0,0,9.6875,M,Q
3,3,male,27.0,0,0,8.6625,M,S
4,3,female,22.0,1,1,12.2875,M,S


In [7]:
#Slicing data, preparing it for Logistic Regression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
X = df_logreg_train.iloc[:, 1:].values #Disregarding "Survived" Columnsss
X_test = df_logreg_test.iloc[:, 0:].values
y = df_logreg_train.iloc[:, 0].values
X_test[0]

array([3, 'male', 34.5, 0, 0, 7.8292, 'M', 'Q'], dtype=object)

In [8]:
#cabin encoding
label_encoder = LabelEncoder()
print(X[:, 6])
X[: , 6] = label_encoder.fit_transform(X[: , 6])
X[: , 6]
X_test[: , 6] = label_encoder.fit_transform(X_test[: , 6])

['M' 'C85' 'M' 'C123' 'M' 'M' 'E46' 'M' 'M' 'M' 'G6' 'C103' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'D56' 'M' 'A6' 'M' 'M' 'M' 'C23 C25 C27' 'M' 'M' 'M'
 'B78' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'D33' 'M' 'B30' 'C52' 'M' 'M' 'M' 'M' 'M' 'B28' 'C83' 'M' 'M'
 'M' 'F33' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'F G73' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'C23 C25 C27' 'M' 'M' 'M' 'E31' 'M' 'M' 'M' 'A5'
 'D10 D12' 'M' 'M' 'M' 'M' 'D26' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'C110' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'B58 B60' 'M' 'M' 'M' 'M' 'E101' 'D26' 'M' 'M' 'M'
 'F E69' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'D47' 'C123' 'M' 'B86' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'F2' 'M' 'M' 'C2' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'E33' 'M' 'M' 'M' 'B19' 'M' 'M' 'M' 'A7' 'M' 'M' 'C49' 'M'
 'M' 'M' 'M' 'M' 'F4' 'M' 'A32' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'F2' 'B4' 'B80'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'G6' 'M' 'M' 'M' 'A31' 'M' 'M' 'M' 'M'
 'M' 'D36' 'M' 'M' 'D15'

In [9]:
#embarked encoding
label_encoder = LabelEncoder()
print(X[:, 7])
X[: , 7] = label_encoder.fit_transform(X[: , 7])
X[: , 7]
label_encoder = LabelEncoder()
X_test[: , 7] = label_encoder.fit_transform(X_test[: , 7])

['S' 'C' 'S' 'S' 'S' 'Q' 'S' 'S' 'S' 'C' 'S' 'S' 'S' 'S' 'S' 'S' 'Q' 'S'
 'S' 'C' 'S' 'S' 'Q' 'S' 'S' 'S' 'C' 'S' 'Q' 'S' 'C' 'C' 'Q' 'S' 'C' 'S'
 'C' 'S' 'S' 'C' 'S' 'S' 'C' 'C' 'Q' 'S' 'Q' 'Q' 'C' 'S' 'S' 'S' 'C' 'S'
 'C' 'S' 'S' 'C' 'S' 'S' 'C' 'S' 'S' 'S' 'C' 'C' 'S' 'S' 'S' 'S' 'S' 'S'
 'S' 'C' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'Q' 'S' 'S' 'S' 'S' 'S' 'S' 'S'
 'S' 'S' 'S' 'S' 'S' 'S' 'C' 'C' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S'
 'S' 'Q' 'S' 'C' 'S' 'S' 'C' 'S' 'Q' 'S' 'C' 'S' 'S' 'S' 'C' 'S' 'S' 'C'
 'Q' 'S' 'C' 'S' 'C' 'S' 'S' 'S' 'S' 'C' 'S' 'S' 'S' 'C' 'C' 'S' 'S' 'Q'
 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'C' 'Q' 'S' 'S' 'S' 'S' 'S'
 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'Q' 'S' 'S' 'C' 'S' 'S' 'C' 'S' 'S'
 'S' 'C' 'S' 'S' 'S' 'S' 'Q' 'S' 'Q' 'S' 'S' 'S' 'S' 'S' 'C' 'C' 'Q' 'S'
 'Q' 'S' 'S' 'S' 'S' 'C' 'S' 'S' 'S' 'C' 'Q' 'C' 'S' 'S' 'S' 'S' 'Q' 'C'
 'S' 'S' 'C' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S'
 'S' 'S' 'S' 'S' 'S' 'S' 'C' 'Q' 'S' 'S' 'C' 'Q' 'S

In [10]:
#Sex encoding
label_encoder = LabelEncoder()
print(X[:, 1])
X[: , 1] = label_encoder.fit_transform(X[: , 1])
X_test[: , 1] = label_encoder.fit_transform(X_test[: , 1])
X[: , 1]

['male' 'female' 'female' 'female' 'male' 'male' 'male' 'male' 'female'
 'female' 'female' 'female' 'male' 'male' 'female' 'female' 'male' 'male'
 'female' 'female' 'male' 'male' 'female' 'male' 'female' 'female' 'male'
 'male' 'female' 'male' 'male' 'female' 'female' 'male' 'male' 'male'
 'male' 'male' 'female' 'female' 'female' 'female' 'male' 'female' 'female'
 'male' 'male' 'female' 'male' 'female' 'male' 'male' 'female' 'female'
 'male' 'male' 'female' 'male' 'female' 'male' 'male' 'female' 'male'
 'male' 'male' 'male' 'female' 'male' 'female' 'male' 'male' 'female'
 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'female' 'male' 'male'
 'female' 'male' 'female' 'female' 'male' 'male' 'female' 'male' 'male'
 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'female' 'male' 'female'
 'male' 'male' 'male' 'male' 'male' 'female' 'male' 'male' 'female' 'male'
 'female' 'male' 'female' 'female' 'male' 'male' 'male' 'male' 'female'
 'male' 'male' 'male' 'female' 'male' 'male' 'male' 'ma

array([1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 0,

In [11]:
#Applying Logistic Regression
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X,y.ravel())
y_predicted = classifier.predict(X_test)
y_predicted

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

In [12]:
print(X[0])
print('*****************')
print(X_test[0])

[3 1 22.0 1 0 7.25 146 2]
*****************
[3 1 34.5 0 0 7.8292 76 1]


In [13]:
# Writing results to csv
titanic_log_reg_test_copy = test_original.copy()
titanic_log_reg_test_copy.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [26]:
titanic_test_passenger_id_array = titanic_log_reg_test_copy.iloc[:,0].values
import csv
#writer = csv.writer(open("./data/logisticRegressionResults.csv", 'w'))
entries =[]
entries.append(["PassengerId","Survived"])
for i in range(y_predicted.size):
    entries.append([titanic_test_passenger_id_array[i],y_predicted[i]])
for i in entries:
    file = open('./data/logisticRegressionResults.csv', 'a', newline='')
    write = csv.writer(file, delimiter = ",")
    write.writerow(i)
    file.close()

In [30]:
#Applying Naive Bayes.
from sklearn.naive_bayes import GaussianNB
gnb_classifier = GaussianNB()
# y values need to be raveled.
gnb_classifier.fit(X,y.ravel())
gnb_predicted = gnb_classifier.predict(X_test)
entries =[]
entries.append(["PassengerId","Survived"])
for i in range(y_predicted.size):
    entries.append([titanic_test_passenger_id_array[i],gnb_predicted[i]])
for i in entries:
    file = open('./data/NBResults.csv', 'a', newline='')
    write = csv.writer(file, delimiter = ",")
    write.writerow(i)
    file.close()